In [1]:
import boto3
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import datetime
import pandas as pd
pd.set_option('display.max_columns', None)
import statistics
import requests
from bs4 import BeautifulSoup

from thefuzz import process
import re
pd.set_option("display.max_rows", None)


In [ ]:
with open("AWS_credentials.txt", "r") as creds:
    creds_list = json.loads(creds.read())
s3 = boto3.resource('s3', aws_access_key_id=creds_list[0], aws_secret_access_key=creds_list[1])
my_bucket = s3.Bucket('hadis-s3-project-bucket')
for object_summary in my_bucket.objects.filter(Prefix="Real-Estate/input/"):
    print(object_summary.key)
    if "Real-Estate/input/"+str(datetime.date.today()) in object_summary.key:
        my_bucket.download_file(object_summary.key, object_summary.key.split("/")[-1])
        print("downloaded")

#manual download  ---> my_bucket.download_file("Real-Estate/input/2024-04-30_6901.parquet.gzip", "2024-04-30_6901.parquet.gzip")

In [2]:
my_spark = SparkSession.builder.getOrCreate()
print(my_spark.catalog.listTables())

[]


In [20]:
path = '2024-06-18_bayut_listings_for_6901.parquet.gzip'
df = my_spark.read.format('parquet').options(header=True,inferSchema=True).load(path)
unreq_cols = ["productLabel","rentFrequency","projectNumber","title_l1","title_l2","title_l3","coverPhoto","photoCount","videoCount","panoramaCount","randBoostScore","randBoostScore_l1",
              "randBoostScore_l2","randBoostScore_l3","floorPlanID","amenities_l1","amenities_l2","amenities_l3","indyScore","indyScore_l1","indyScore_l2","indyScore_l3","photoIDs",
              "coverVideo","slug_l1","slug_l2","slug_l3","category","_highlightResult","id","ownerID","userExternalID","sourceID","state","_geoloc","purpose","product","productScore","referenceNumber",
              "permitNumber","title","externalID","slug","score","score_l1","score_l2","score_l3","hash","isVerified","verification","veridiedScore","type","ownerAgent","amenities","citylevelScore",
              "hasMatchingFloorPlans","hidePrice","locationPurposeTier","plotArea","objectID","project","offplanDetails","completionDetails","paymentPlanSummaries","hasTransactionHistory","hasDldBuildingNk",
              "availabilityStatus","occupancyStatus","keywords","verifiedScore"]
x = datetime.datetime.timestamp(datetime.datetime.now() - datetime.timedelta(days=600))
df = df.drop(*unreq_cols).filter((df.createdAt > x) | (df.updatedAt > x))

df = df.withColumn("createdAt", to_timestamp(col("createdAt").cast("timestamp"), "yyyyMMddHHmmSS"))
df = df.withColumn("updatedAt", to_timestamp(col("updatedAt").cast("timestamp"), "yyyyMMddHHmmSS"))
df = df.withColumn("reactivatedAt", to_timestamp(col("reactivatedAt").cast("timestamp"), "yyyyMMddHHmmSS"))

df = df.withColumn("loc_size",size("location"))
df = df.withColumn("country", lower(col("location")[0]["name"]))
df = df.withColumn("city", lower(col("location")[1]["name"]))
df = df.withColumn("neighborhood", lower(col("location")[2]["name"]))
df = df.withColumn("neighborhood", regexp_replace(col("neighborhood"),"[\\s]","-"))
df = df.withColumn("neighborhood", regexp_replace(col("neighborhood"),"[\\.]",""))
df = df.withColumn('sub_neighborhood', lower(when(col('loc_size') == 6, col("location")[3]["name"]).otherwise(lit(None))))
df = df.withColumn("sub_neighborhood", regexp_replace(col("sub_neighborhood"),"[\\s]","-"))
df = df.withColumn("sub_neighborhood", regexp_replace(col("sub_neighborhood"),"[\\.]",""))
df = df.withColumn('building_series', lower(when(col('loc_size') >= 5, element_at(col('location'), -2)["name"]).otherwise(lit(None))))
df = df.withColumn("building_series", regexp_replace(col("building_series"),"[\\s]","-"))
df = df.withColumn("building_series", regexp_replace(col("building_series"),"[\\.]",""))
df = df.withColumn("building_name", element_at(col('location'), -1)["name"])
# df = df.withColumn("building_name", lower(element_at(col('location'), -1)["name"]))
# df = df.withColumn("building_name", regexp_replace(col("building_name"),"[\\s]","-"))
# df = df.withColumn("building_name", regexp_replace(col("building_name"),"[\\.]",""))

df = df.drop("loc_size","location")

df = df.withColumn("phone_number", col("phoneNumber")["mobile"])
df = df.withColumn("whatsapp", col("phoneNumber")["whatsapp"])
df = df.drop("phoneNumber")

df = df.withColumn("agency", col("agency")["name"])

df = df.withColumn("geography_lat",col("geography")["lat"])
df = df.withColumn("geography_lat",round(df["geography_lat"],3))
df = df.withColumn("geography_lng",col("geography")["lng"])
df = df.withColumn("geography_lng",round(df["geography_lng"],3))
df = df.drop("geography")

df = df.withColumn("area",col("area")*10.7639)

In [21]:
def add_listing(units_in_building):
    units_in_building = units_in_building.reset_index(drop=True)
    listings_cols = ["price","contactName","agency","phone_number","whatsapp","createdAt","updatedAt","reactivatedAt"]
    listings = {}
    for index, unit_in_building in units_in_building.iterrows():
        listing = {}
        for listing_col in listings_cols:
            listing[listing_col] = unit_in_building[listing_col]
        listings["listing_"+str(index+1)] = listing
    return listings

pdf = df.select("*").toPandas()
pdf.sort_values("building_name",inplace = True, ignore_index=True)

pdf_new = pd.DataFrame(columns=pdf.columns)
pdf_new.rename(columns = {'price':'price_avg', 'area':'area_avg'}, inplace = True)
shared_cols = ["building_name","building_series","sub_neighborhood","neighborhood","furnishingStatus","completionStatus","rooms","baths","geography_lat","geography_lng","city","country"]
pdf_new = pdf_new.drop(columns=["createdAt","updatedAt","reactivatedAt","contactName","agency","phone_number","whatsapp"])
pdf_new["listings"] = ""
pdf_new["num_listings"] = ""

for units_in_building in pdf.groupby(["building_name"]):
    units_in_building = units_in_building[1].reset_index(drop=True)

    units_range = list(range(len(units_in_building)))
    for i in units_range:
        similar_listings = [i]
        for j in range(i+1,len(units_in_building)):
            if (((units_in_building.iloc[j]["price"] <= (units_in_building.iloc[i]["price"]+200000)) and (units_in_building.iloc[j]["price"] >= (units_in_building.iloc[i]["price"]-200000))) and
                ((units_in_building.iloc[j]["area"] <= (units_in_building.iloc[i]["area"]+10)) and (units_in_building.iloc[j]["area"] >= (units_in_building.iloc[i]["area"]-10))) and
                (units_in_building.iloc[j]["furnishingStatus"] == units_in_building.iloc[i]["furnishingStatus"]) and
                (units_in_building.iloc[j]["baths"] == units_in_building.iloc[i]["baths"])):
                similar_listings.append(j)
                if j in units_range:
                    units_range.remove(j)

        # new_row = pd.DataFrame()
        new_row =  units_in_building.iloc[i:i+1][shared_cols]
        new_row["price_avg"] = statistics.mean(units_in_building.iloc[similar_listings]["price"])
        new_row["area_avg"] = statistics.mean(units_in_building.iloc[similar_listings]["area"])
        listings = add_listing(units_in_building.iloc[similar_listings])
        new_row["listings"] = [listings]
        new_row["num_listings"] = len(listings)
        
        pdf_new = pd.concat([pdf_new,new_row],ignore_index=True)


pdf_new.head()

C:\Users\hadi hashemi\AppData\Local\Temp\ipykernel_17240\4230664365.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pdf_new = pd.concat([pdf_new,new_row],ignore_index=True)


,price_avg,rooms,baths,area_avg,furnishingStatus,completionStatus,deal,country,city,neighborhood,sub_neighborhood,building_series,building_name,geography_lat,geography_lng,listings,num_listings
0,2.600000e+07,4,5,6649.993565,unfurnished,completed,NaN,uae,dubai,downtown-dubai,None,mohammad-bin-rashid-boulevard,118 Downtown,25.197,55.279,"{'listing_1': {'price': 26000000.0, 'contactNa...",1
1,4.107294e+06,2,2,1390.998654,unfurnished,under-construction,NaN,uae,dubai,downtown-dubai,None,None,25H Heimat,25.191,55.270,"{'listing_1': {'price': 4107294.0, 'contactNam...",1
2,3.966667e+06,2,3,1389.998655,furnished,under-construction,NaN,uae,dubai,downtown-dubai,None,None,25H Heimat,25.191,55.270,"{'listing_1': {'price': 4100000.0, 'contactNam...",3
3,2.411580e+06,1,2,763.499261,unfurnished,under-construction,NaN,uae,dubai,downtown-dubai,None,None,25H Heimat,25.191,55.270,"{'listing_1': {'price': 2400000.0, 'contactNam...",2
4,5.596250e+06,3,4,1955.998107,unfurnished,under-construction,NaN,uae,dubai,downtown-dubai,None,None,25H Heimat,25.191,55.270,"{'listing_1': {'price': 5596250.0, 'contactNam...",1


In [54]:
path_2 = '2024-06-18_DLD_transactions.parquet.gzip'
dld = my_spark.read.format('parquet').options(header=True,inferSchema=True).load(path_2)
dld_columns_keep = ["instance_date","building_name_en","procedure_area","actual_worth","rooms_en","area_name_en","master_project_en","procedure_name_en","trans_group_en",
               "property_type_en","property_sub_type_en","property_usage_en","reg_type_en"]
dld = dld.select(*dld_columns_keep)
dld = dld.withColumn("instance_date", to_timestamp(col("instance_date"),"dd-MM-yyyy"))
dld = dld.sort(dld.instance_date.desc())

dld = dld.filter(dld.property_type_en == "Unit").filter("procedure_name_en like '%Sell%'") #.filter(dld.rooms_en == "2 B/R").filter(dld.property_usage_en == "Residential")
dld = dld.filter(dld.area_name_en == "Burj Khalifa")
dld = dld.withColumn("procedure_area",col("procedure_area")*10.7639)
dld = dld.withColumn("procedure_area",format_number(dld.procedure_area, 0))

dld.show(5)
# pdld = dld.select("*").toPandas()

+-------------------+--------------------+--------------+------------+--------+------------+-----------------+--------------------+--------------+----------------+--------------------+-----------------+-------------------+
|      instance_date|    building_name_en|procedure_area|actual_worth|rooms_en|area_name_en|master_project_en|   procedure_name_en|trans_group_en|property_type_en|property_sub_type_en|property_usage_en|        reg_type_en|
+-------------------+--------------------+--------------+------------+--------+------------+-----------------+--------------------+--------------+----------------+--------------------+-----------------+-------------------+
|2024-06-13 00:00:00|EXQUISITE LIVING ...|         1,658|   4133832.0|   2 B/R|Burj Khalifa|     Burj Khalifa|Sell - Pre regist...|         Sales|            Unit|                Flat|      Residential|Off-Plan Properties|
|2024-06-13 00:00:00|The St. Regis Res...|         1,269|   3480000.0|   2 B/R|Burj Khalifa|     Burj Khalif

In [ ]:
# groups = [x[0] for x in dld.select("building_name_en").distinct().collect()]
# groups_list = [dld.filter(col("building_name_en")==x) for x in groups]

# for x in groups_list:
#     groups_2 = [y[0] for y in x.select("procedure_area").distinct().collect()]
#     groups_list_2 = [x.filter(col("procedure_area")==y) for y in groups_2]
#     for y in groups_list_2:
#         y.show()
#     break

In [6]:
def make_df_for_comparing(org_df):
    residences = ["THE RESIDENCES II - TOWER 1","Residences_East Poduim ","Residences_E3","Residences - T3","THE RESIDENCES II - TOWER 3","Residences_West Poduim ",
            "THE RESIDENCES II - TOWER 2","Residences_E2","Residences_W3","Residences_W1","THE DUBAI MALL RESIDENCES","THE RESIDENCES II - BOULEVARD VILLAS",
            "The Residence 1","The Residence 2","The Residence 3","The Residence 4","The Residence 5","The Residence 6","The Residence 7","The Residence 8",
            "The Residence 9","W Residences","25H Heimat"]
    org_df = org_df.loc[~org_df.iloc[:,0].isin(residences)]
    new_df = pd.DataFrame()
    new_df["building_name"] = org_df
    new_df["lower_building_name"] = new_df["building_name"].str.lower()
    new_df["tower_in_name"] = ""
    new_df["tower_number"] = ""
    new_df["name_for_comparing"] = ""
    new_df["name_&_number_for_comparing"] = ""
    for index, row in new_df.iterrows():
        row["lower_building_name"] = row["lower_building_name"].replace("blvd", "boulevard")
        sep_name = row["lower_building_name"].split()
        if sep_name[0] == "bd":
            sep_name = sep_name[1:]
        
        if sep_name[-1] == "a":
            sep_name[-1] = str(1)
        elif sep_name[-1] == "b":
            sep_name[-1] = str(2)
        elif sep_name[-1] == "c":
            sep_name[-1] = str(3)
        
        if sep_name[-1][-1].isdigit() or (sep_name[-1] == "podium"):
            if sep_name[-1].isdigit():
                new_df.loc[index,"tower_number"] = sep_name[-1]
                new_df.loc[index,"name_for_comparing"] = " ".join(sep_name[:-1])
            elif "t" in sep_name[-1]:
                new_df.loc[index,"tower_in_name"] = True
                new_df.loc[index,"tower_number"] = "".join(re.findall(r'\d+', sep_name[-1]))
                new_df.loc[index,"name_for_comparing"] = " ".join(sep_name[:-1])
            else:
                new_df.loc[index,"tower_number"] = sep_name[-1]
                new_df.loc[index,"name_for_comparing"] = " ".join(sep_name[:-1])
            sep_name = sep_name[:-1]
        else:
            new_df.loc[index,"tower_number"] = False
        
        if (new_df.loc[index,"tower_in_name"] == True) or (("tower" in sep_name) or ("t" in sep_name)):
            new_df.loc[index,"tower_in_name"] = True
            name_for_comparing = [x for x in sep_name if (x!="tower" and x!="t")]
            new_df.loc[index,"name_for_comparing"] = " ".join(name_for_comparing)
        else:
            new_df.loc[index,"tower_in_name"] = False
        
        if (new_df.loc[index,"tower_in_name"]==False) and (new_df.loc[index,"tower_number"]==False):
            new_df.loc[index,"name_for_comparing"] = row["lower_building_name"]
        
        if (new_df.loc[index,"tower_number"]!=False) and (new_df.loc[index,"tower_number"]!="podium"):
            new_df.loc[index,"name_&_number_for_comparing"] = new_df.loc[index,"name_for_comparing"] +" "+ str(new_df.loc[index,"tower_number"])
        else:
            new_df.loc[index,"name_&_number_for_comparing"] = new_df.loc[index,"name_for_comparing"]
    new_df = new_df.drop('lower_building_name', axis=1)
    return new_df

In [58]:
bayut_building_names = pd.DataFrame()
bayut_building_names["building_name"] = pdf["building_name"].unique()
bayut_building_names = make_df_for_comparing(bayut_building_names)

dld_building_names = dld.select("building_name_en").distinct().toPandas()
dld_building_names.rename(columns = {'building_name_en':'building_name'}, inplace = True)
dld_building_names = make_df_for_comparing(dld_building_names)
dld_building_names.sort_values("name_&_number_for_comparing",inplace = True, ignore_index=True)

In [79]:
matched_names = pd.DataFrame()
matched_names["bayut_name"] = bayut_building_names["building_name"]
matched_names["bayut_name_compared"] = ""
matched_names["dld_name_compared"] = ""
matched_names["dld_name"] = ""

for index, row in bayut_building_names.iterrows():
    # print("checked name: "+row["name_for_comparing"])
    # print("true name: "+row["building_name"])
    matched_names.loc[index, "bayut_name_compared"] = row["name_for_comparing"]
    matches = pd.DataFrame(process.extract(row["name_for_comparing"], dld_building_names["name_for_comparing"],limit=10))
    # print(matches)
    matches = matches.loc[matches[1]>=67]
    final_match = process.extract(row["name_&_number_for_comparing"], dld_building_names.loc[matches[2],"name_&_number_for_comparing"],limit=5)
    # print(final_match)
    if final_match:
        matched_names.loc[index, "dld_name"] = [dld_building_names.loc[final_match[0][2],"building_name"]]
        matched_names.loc[index, "dld_name_compared"] = dld_building_names.loc[final_match[0][2],"name_&_number_for_comparing"]

matched_names = matched_names.reset_index(drop=True)
matched_names_fixes = {"Burj Views A":["Burj ViewsTower A"],
                       "The Lofts Central Tower":["LOFTS T CENT"],
                       "Tajer Residence":["ALTAJER"],
                       "St. Regis The Residences":["The St. Regis Residences - Tower 1","The St. Regis Residences - Tower 2"],
                       "Vida Residence Downtown":["Vida Dubai Mall Tower 2"],
                       "29 Boulevard":["BD 29 BLVD PODIUM","29 BLVD T1","BD 29 BLVD T2"],
                       "BLVD Heights":["Blvd Heights Podium","Blvd Heights T1","Blvd Heights T2"],
                       "Boulevard Central":["BLVD CENTRAL PODIUM","BLVD CENTRAL 1","BLVD CENTRAL 2"],
                       "Boulevard Crescent Towers":["BLVD CRESCENT Podium","BLVD CRESCENT Tower 1","BLVD CRESCENT Tower 2"],
                       "Burj Views":["Burj ViewsTower A","Burj Views Tower B","Burj Views Tower C","Burj Views Podium"],
                       "118 Downtown":["THE 118"],
                       "Act One | Act Two Towers":["Act One Act Two Tower 1","Act One Act Two Tower 2"],
                       "Act Two":["Act One Act Two Tower 2"],
                       "Address Dubai Mall":[],
                       "Armani Hotel and Residences":["Burj Khalifa"],
                       "Baccarat Hotel And Residences":["Baccarat Residence T1"],
                       "Burj Al Nujoom":["Al NOUJOUM TOWER"],
                       "Claren Podium":["Claren PD"],
                       "DT1 Tower":["DT1"],
                       "Downtown Dubai":[],
                       "Forte":["Forte T1","Forte T2"],
                       "Kempinski Central Avenue Dubai":["THE DUBAI MALL RESIDENCES"],
                       "Kempinski The Boulevard":[],
                       "Miska":["MISKA 1","MISKA 2","MISKA 3","MISKA 4","MISKA 5"],
                       "Standpoint Towers":["BD Standpoint A","BD Standpoint B"],
                       "The Address Fountain Views 3":["Address Fountain Views Hotel"],
                       "The Address Residences Dubai Opera":["The Address Residences Dubai Opera T1","The Address Residences Dubai Opera T2"],
                       "The Address Sky View Tower 1":["Address Residences Sky View"],
                       "The Address Sky View Tower 2":["BRIDGE SKY"],
                       "The Lofts":["LOFTS T CENT","LOFTS T EAST","LOFTS T WEST","LOFTS"],
                       "The Residence Burj Khalifa":["Burj Khalifa Residences"],
                       "The Residences":["Residences_W3","Residences_W1","Residences_E1","Residences_E2","Residences_E3","THE RESIDENCES II - TOWER 3","THE RESIDENCES II - TOWER 1","THE RESIDENCES II - TOWER 2"],
                       "Vida Dubai Mall":["Vida Dubai Mall Tower 1","Vida Dubai Mall Tower 2"],
                       "Vida Residence Downtown":["VIDA RESIDENCE DOWNTOWN"],
                       "Souk Al Bahar":["ALBAHAR"]
                       }
for key in matched_names_fixes:
    matched_names.at[*np.where(matched_names["bayut_name"]==key)[0],"dld_name"] = matched_names_fixes[key]

residences_matched_names_fixes = {"bayut_name" : ["W Residences","The Residence 1","The Residence 3","The Residence 4","The Residence 5","The Residence 6","The Residence 7",
                                                  "The Residence 8","The Residence 9"],
                                "dld_name" : [["City Center Residences"],["Residences_W3"],["Residences_W1"],["Residences_E1"],["Residences_E2"],["Residences_E3"],["THE RESIDENCES II - TOWER 3"],
                                              ["THE RESIDENCES II - TOWER 1"],["THE RESIDENCES II - TOWER 2"]]}
residences_matched_names_fixes = pd.DataFrame(data=residences_matched_names_fixes)
residences_matched_names_fixes["bayut_name_compared"] = ""
residences_matched_names_fixes["dld_name_compared"] = ""
matched_names = pd.concat([matched_names,residences_matched_names_fixes],ignore_index=True)

In [ ]:
residences = ["Residences_East Poduim ","Residences_West Poduim ","THE RESIDENCES II - BOULEVARD VILLAS",
              "THE DUBAI MALL RESIDENCES","Residences - T3","Residences_W1",
              "25H Heimat"]

In [80]:
temp = dld.select("building_name_en").distinct().toPandas()
# temp = pd.DataFrame(data=pdf["building_name"].unique(),columns=["name"])

found_buildings = []
for row in matched_names["dld_name"]:
    if row:
        found_buildings.append(row[0])

temp[~temp["building_name_en"].isin(found_buildings)]

,building_name_en
3,Emaar Square Bldg 1
7,Emaar Square Bldg 4
11,REEHAN 8
12,AL MANZIL HOTEL
17,THE RESIDENCES II - LAKE VILLAS
24,SOUTH RIDGE - PODIUM VILLAS WEST
28,BD BLVD Plaza T2
32,Residences_East Poduim
40,Emaar Square Bldg 2
60,Residences - T3


In [91]:
pdld.loc[pdld["building_name_en"]=="Residences - T3"].head()

,instance_date,building_name_en,procedure_area,actual_worth,rooms_en,area_name_en,master_project_en,procedure_name_en,trans_group_en,property_type_en,property_sub_type_en,property_usage_en,reg_type_en
45228,2008-09-08,Residences - T3,"1,452",4900000.0,2 B/R,Burj Khalifa,Burj Khalifa,Sell,Sales,Unit,Flat,Residential,Existing Properties
45229,2008-09-08,Residences - T3,"1,452",3900000.0,2 B/R,Burj Khalifa,Burj Khalifa,Sell,Sales,Unit,Flat,Residential,Existing Properties


In [ ]:
pdf["building_name"].unique()

In [81]:
matched_names

,bayut_name,bayut_name_compared,dld_name_compared,dld_name
0,118 Downtown,118 downtown,burj lake hotel - the address downtown,[THE 118]
1,29 Boulevard,29 boulevard,29 boulevard,"[BD 29 BLVD PODIUM, 29 BLVD T1, BD 29 BLVD T2]"
2,29 Boulevard 1,29 boulevard,29 boulevard 1,[29 BLVD T1]
3,29 Boulevard 2,29 boulevard,29 boulevard 2,[BD 29 BLVD T2]
4,29 Boulevard Podium,29 boulevard,29 boulevard,[BD 29 BLVD PODIUM]
5,8 Boulevard Walk,8 boulevard walk,8 boulevard walk,[8 BLVD WALK]
6,Act One,act one,act one act two 1,[Act One Act Two Tower 1]
7,Act One | Act Two Towers,act one | act two towers,act one act two 1,"[Act One Act Two Tower 1, Act One Act Two Towe..."
8,Act Two,act two,act one act two 1,[Act One Act Two Tower 2]
9,Address Dubai Mall,address dubai mall,address fountain views hotel,[]


In [64]:
pdld=dld.toPandas()

In [72]:
for building in pdld["building_name_en"].unique():
    building = pdld[pdld["building_name_en"]==building].reset_index()
    if building.loc[0,"actual_worth"] == 5100000:
        print(building.iloc[0,:])

index                                           114
instance_date                   2024-06-06 00:00:00
building_name_en        Address Residences Sky View
procedure_area                                1,519
actual_worth                              5100000.0
rooms_en                                      2 B/R
area_name_en                           Burj Khalifa
master_project_en                      Burj Khalifa
procedure_name_en                              Sell
trans_group_en                                Sales
property_type_en                               Unit
property_sub_type_en                Hotel Apartment
property_usage_en                      Hospitality 
reg_type_en                     Existing Properties
Name: 0, dtype: object


In [92]:
pdf_new.loc[pdf_new["building_name"]=="Address Dubai Mall"]

,price_avg,rooms,baths,area_avg,furnishingStatus,completionStatus,deal,country,city,neighborhood,sub_neighborhood,building_series,building_name,geography_lat,geography_lng,listings,num_listings
178,3000000.0,1,2,841.999185,furnished,completed,NaN,uae,dubai,downtown-dubai,None,None,Address Dubai Mall,25.195,55.282,"{'listing_1': {'price': 3000000.0, 'contactNam...",1
